In [4]:
import chromadb 

host='localhost'
port='8000'
client = chromadb.HttpClient(host=host, port=port) 


client.list_collections()

[Collection(name=temp), Collection(name=langchain), Collection(name=resume)]

In [5]:
data =  {
        "source": {
            "id": None,
            "name": "Threadreaderapp.com"
        },
        "author": None,
        "title": "Walls are starting to close in for Tesla, let's have a closer look",
        "description": "@ZeroSumBond: $TSLA Walls are starting to close in for Tesla, let's have a closer look: 1) The recent turnover in senior executives is incredible, with most having a tenure well over 5 years. From CFO, to...\u2026",
        "url": "https://threadreaderapp.com/thread/1787501413137858617.html",
        "urlToImage": "https://threadreaderapp.com/images/screenshots/thread/1787501413137858617.jpg",
        "publishedAt": "2024-05-22T20:22:25Z",
        "content": "Support us! We are indie developers!\r\nThis site is made by just two indie developers on a laptop doing marketing, support and development! Read more about the story.\r\nBecome a Premium Member ($3/mont\u2026 [+395 chars]"
    }

In [12]:
from api_test import Perigon
import os 
import json 
api = Perigon(os.getenv('PERIGON_API_KEY'), 'https://api.goperigon.com/v1/')
# rom=2024-05-03&q="SpaceX" OR "Tesla"&sourceGroup=top100&language=en&apiKey=[KEY]
params = { 
	"from": "2024-05-03",
	"sourceGroup": "top100",
	"language": "en"
}
res = api.all('SpaceX', **params)


https://api.goperigon.com/v1/all?apiKey=6b47d882-a23a-44e1-b615-88c8b833512e&q=SpaceX&from=2024-05-03&sourceGroup=top100&language=en
Successfully Retrieve 1024 articles


In [13]:
print(res['articles'][0]['url'])
print(res['articles'][0]['content'])
print(res['articles'][0]['summary'])

print(res['articles'][0])
f = open('temp.json', 'w')
json.dump(res,f, indent=4, ensure_ascii=True)
f.close()

https://www.bloomberg.com/news/features/2024-05-21/how-to-buy-spacex-stock-here-s-a-pre-ipo-investing-guide?srnd=homepage-europe
The constellation Monoceros, faintly visible against the Milky Way, has for centuries occupied the heavens as “the unicorn.” The other space unicorn? Elon Musk ’s Space Exploration Technologies Corp. ... [1005 symbols]
Elon Musk's Space Exploration Technologies Corp. (SpaceX), the most highly valued startup in the US, has grown into a $180 billion colossus that launches satellites, operates the Starlink internet service, and is a critical NASA subcontractor. However, SpaceX remains privately held and its shares do not currently trade on a stock exchange. As a result, obtaining a piece of SpaceX is a process that can increase its valuation. Investing in SpaceX is not simple, but it can create demand and raise the company's valuation.
{'url': 'https://www.bloomberg.com/news/features/2024-05-21/how-to-buy-spacex-stock-here-s-a-pre-ipo-investing-guide?srnd=homepa

In [2]:
from langchain_community.vectorstores import Chroma
import chromadb.utils.embedding_functions as embedding_functions
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import CharacterTextSplitter

In [54]:
text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=5)

documents = text_splitter.create_documents([res['articles'][0]['summary']],metadatas=[{'url': res['articles'][0]['url']}])

docs = text_splitter.split_documents(documents)

In [56]:
print(docs)

[Document(page_content="Elon Musk's Space Exploration Technologies Corp. (SpaceX), the most highly valued startup in the US, has grown into a $180 billion colossus that launches satellites, operates the Starlink internet service, and is a critical NASA subcontractor. However, SpaceX remains privately held and its shares do not currently trade on a stock exchange. As a result, obtaining a piece of SpaceX is a process that can increase its valuation. Investing in SpaceX is not simple, but it can create demand and raise the company's valuation.", metadata={'url': 'https://www.bloomberg.com/news/features/2024-05-21/how-to-buy-spacex-stock-here-s-a-pre-ipo-investing-guide?srnd=homepage-europe'})]


In [48]:
embedding_function = OllamaEmbeddings(model="nomic-embed-text")

db = Chroma.from_documents(docs, embedding_function, client=client, collection_name='temp')

In [50]:
db.similarity_search('elon') 

[Document(page_content="Elon Musk's Space Exploration Technologies Corp. (SpaceX), the most highly valued startup in the US, has grown into a $180 billion colossus that launches satellites, operates the Starlink internet service, and is a critical NASA subcontractor. However, SpaceX remains privately held and its shares do not currently trade on a stock exchange. As a result, obtaining a piece of SpaceX is a process that can increase its valuation. Investing in SpaceX is not simple, but it can create demand and raise the company's valuation.", metadata={'url': 'https://www.bloomberg.com/news/features/2024-05-21/how-to-buy-spacex-stock-here-s-a-pre-ipo-investing-guide?srnd=homepage-europe'})]

In [2]:
from APItoVDB import APItoVDB
import json 
from langchain_community.embeddings import OllamaEmbeddings

db = APItoVDB()

# choose your embedding models 
embedding_function = OllamaEmbeddings(model="nomic-embed-text")

with open('temp.json', 'r') as f: 
	res = json.load(f)

# print(res)

db.delete('temp') 
db.create('temp', emb_fn=embedding_function)


for article in res['articles'][:100]:
	if not article['summary']:
		continue
	# print(article['summary'])
	# print(article['url'])
	metadata = {'url': article['url']}
	db.load(article['summary'], metadata) 



print(db.query("TSLA"))


temp is Deleted
temp is Created
[Document(page_content="In the June 2024 issue, we explore the ways that artificial intelligence is shaping our economy and culture. The article discusses the delays in Artemis II and Artemis III, a crewed lunar landing mission, and the potential cost of these missions. In January, NASA announced that it would be delayed until September 2025, a year later than originally planned. The project continues to balloon in price and is expected to increase further. Other delays include Lockheed Martin's Orion capsule, which burned more than anticipated during a reentry test. The Federal Aviation Administration (FAA) investigations can add weeks or months between Starship tests. So far, SpaceX has been able to test the spacecraft three times, most recently on March 14. The company hopes to run six Starship test launches this year, but the FAA may delay further tests.", metadata={'url': 'https://www.yahoo.com/tech/spacex-edges-closer-moon-100012160.html'}), Docume